In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Importing required libraries

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import random
import cv2
import math
import numpy as np
import scipy
import scipy.misc
from scipy import pi
from subprocess import call
from datetime import datetime
from itertools import islice
import matplotlib.pyplot as plt 
import tensorflow as tf

In [3]:
# Installing Required Libraries used for unpackin rar file
!pip install pyunpack
!pip install patool

  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
  Stored in directory: /root/.cache/pip/wheels/41/22/19/af15ef6264c58b625a82641ed7483ad05e258fbd8925505227
Successfully built pyunpack easyprocess
    100% |████████████████████████████████| 81kB 3.1MB/s 


In [0]:
# Creating a Directiory to store unpacked dataset

os.mkdir("Driving Data")

In [0]:
from pyunpack import Archive
Archive('drive/My Drive/Autopilot-TensorFlow-master.rar').extractall('Driving Data')

In [0]:
# Dataset Preparation

import scipy.misc
import random

xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("Driving Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("Driving Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


train_xs = xs[:int(len(xs) * 0.7)] # splitting data into 70:30 ratio, as per the task assigned
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


In [7]:

print(num_train_images) # Total Number of Images in Train Data
print(num_val_images)   # Total Number of Images in Test Data


31784
13621


# Linear Activation Function**

In [0]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape = shape, stddev = 0.1)
    return tf.Variable(initial) 

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(previous_input, filter_input, strides):
    return tf.nn.conv2d(previous_input, filter_input, strides = [1, strides, strides, 1], padding = "VALID")
  
  
x_input = tf.placeholder(tf.float32, shape = [None, 66, 200, 3], name = "Plc_1")
y_true = tf.placeholder(tf.float32, name = "Plc_2")

x_image = x_input

In [9]:
# Model Architecture

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.identity(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) 


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
LOGDIR = 'My_Model'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_true, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver()

# op to write logs to Tensorboard
logs_path = 'Driving Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 10
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={x_input: xs, y_true: ys, keep_prob: 0.5})
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={x_input:xs, y_true: ys,keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x_input:xs, y_true: ys, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
        if not os.path.exists(LOGDIR):
            os.makedirs(LOGDIR)
        checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
        filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


Instructions for updating:
Use tf.cast instead.
Epoch: 0, Step: 0, Loss: 6.8046
Epoch: 0, Step: 10, Loss: 5.8396
Epoch: 0, Step: 20, Loss: 5.49062
Epoch: 0, Step: 30, Loss: 5.25492
Epoch: 0, Step: 40, Loss: 5.23264
Epoch: 0, Step: 50, Loss: 4.82687
Epoch: 0, Step: 60, Loss: 4.88862
Epoch: 0, Step: 70, Loss: 4.97269
Epoch: 0, Step: 80, Loss: 4.65724
Epoch: 0, Step: 90, Loss: 4.25892
Epoch: 0, Step: 100, Loss: 4.12881
Epoch: 0, Step: 110, Loss: 3.94583
Epoch: 0, Step: 120, Loss: 3.8538
Epoch: 0, Step: 130, Loss: 4.12429
Epoch: 0, Step: 140, Loss: 4.5821
Epoch: 0, Step: 150, Loss: 3.81564
Epoch: 0, Step: 160, Loss: 4.26803
Epoch: 0, Step: 170, Loss: 3.65803
Epoch: 0, Step: 180, Loss: 4.01458
Epoch: 0, Step: 190, Loss: 3.66808
Epoch: 0, Step: 200, Loss: 3.23379
Epoch: 0, Step: 210, Loss: 3.15084
Epoch: 0, Step: 220, Loss: 3.12654
Epoch: 0, Step: 230, Loss: 3.15744
Epoch: 0, Step: 240, Loss: 4.0511
Epoch: 0, Step: 250, Loss: 3.11754
Epoch: 0, Step: 260, Loss: 5.1035
Epoch: 0, Step: 270, Los

In [11]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "My_Model/model.ckpt")

# img = cv2.imread('steering_wheel_image.jpg', 0) #here, second parameter '0' specifies that img.shape will return only height and
#width of the image and not the number of channels. It is a colored image so number of channels = 3, which it will not return.
# rows, cols = img.shape
degree_radians = []

for i in range(len(val_xs)):
    full_image = cv2.imread(val_xs[i],cv2.IMREAD_COLOR)
#     cv2.imshow('Frame Window', full_image)
    image = ((cv2.resize(full_image[-150:], (200, 66)) / 255.0).reshape((1, 66, 200, 3)))
    degrees = sess.run(y, feed_dict = {x_input: image, keep_prob: 1.0})[0][0] *180 / pi #here, we have converted the
    #predicted degrees from radians to degrees.
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(val_ys[i]*180/scipy.pi) + " (actual)")
#     M = cv2.getRotationMatrix2D((cols/2,rows/2), -degrees, 1) #this function rotate the image by a given degrees.
#     dst = cv2.warpAffine(src = img, M = M, dsize = (cols, rows)) #warpAffine function applies rotation to the image
#     cv2.imshow("Steering Wheel", dst)
    degree_radians.append(degrees)
    



Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from My_Model/model.ckpt


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Steering angle: 2.4468162324141027 (pred)	-28.34 (actual)
Steering angle: 2.4468096156683443 (pred)	-28.840000000000003 (actual)
Steering angle: 2.446814738310222 (pred)	-29.75 (actual)
Steering angle: 2.4468207147257455 (pred)	-31.06 (actual)
Steering angle: 2.446821995386215 (pred)	-32.27 (actual)
Steering angle: 2.4468164458575146 (pred)	-33.48 (actual)
Steering angle: 2.4468162324141027 (pred)	-34.39 (actual)
Steering angle: 2.446831386896324 (pred)	-35.599999999999994 (actual)
Steering angle: 2.446826904584681 (pred)	-36.5 (actual)
Steering angle: 2.4468209281691573 (pred)	-37.61 (actual)
Steering angle: 2.446823062603273 (pred)	-38.62 (actual)
Steering angle: 2.4468245567071536 (pred)	-39.63 (actual)
Steering angle: 2.4468175130745724 (pred)	-39.93000000000001 (actual)
Steering angle: 2.4468134576497524 (pred)	-40.03000000000001 (actual)
Steering angle: 2.4468089753381097 (pred)	-40.03000000000001 (actual)
Steering angle: 2.4468134576497524 (pred)	-40.03000000000001 (actual)
Stee

we are getting the same values for the prections

# Atan Activation Function**

In [0]:
import tensorflow as tf
import scipy

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.atan(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the atan output

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "Driving Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/save/model.ckpt")

# img = cv2.imread('steering_wheel_image.jpg',0)
# rows,cols = img.shape

# smoothed_angle = 0


#read data.txt
xs = []
ys = []
with open("Driving Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("Driving Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.8)
print("Starting frameofvideo:" +str(i))



INFO:tensorflow:Restoring parameters from Driving Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/save/model.ckpt


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Starting frameofvideo:36325


In [0]:
range(i)

range(0, 36325)

In [0]:
for i in range(i):
    full_image = scipy.misc.imread("Driving Data/Autopilot-TensorFlow-master/Autopilot-TensorFlow-master/driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = y.eval(feed_dict={x: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
#     cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
#     smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
#     M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
#     dst = cv2.warpAffine(img,M,(cols,rows))
#     cv2.imshow("steering wheel", dst)
#     i += 1

# cv2.destroyAllWindows()


Steering angle: -0.8550699414508682 (pred)	0.0 (actual)
Steering angle: -0.3295641513333728 (pred)	0.0 (actual)
Steering angle: -0.5430869104845198 (pred)	0.0 (actual)
Steering angle: -0.08143825312455287 (pred)	0.0 (actual)
Steering angle: -0.10819411180558283 (pred)	0.0 (actual)
Steering angle: -0.14139456079342394 (pred)	0.0 (actual)
Steering angle: -0.058452432081745165 (pred)	0.0 (actual)
Steering angle: -0.13026232628186843 (pred)	0.0 (actual)
Steering angle: -0.34179840751904084 (pred)	0.0 (actual)
Steering angle: -0.3033910531968691 (pred)	0.0 (actual)
Steering angle: -0.6926033799540955 (pred)	0.0 (actual)
Steering angle: -1.0950676877637482 (pred)	0.0 (actual)
Steering angle: -0.9184443319125076 (pred)	0.0 (actual)
Steering angle: -0.6834025815335594 (pred)	0.0 (actual)
Steering angle: -0.4852656239006595 (pred)	0.0 (actual)
Steering angle: -0.6929042284426934 (pred)	0.0 (actual)
Steering angle: -0.31883912683043963 (pred)	0.0 (actual)
Steering angle: -0.34943149076207386 (pr

IndexError: ignored

# Conclusion:

1. The linear activation function told us that my steering wheel did not move so it was fixed at a single place.
2. __So we tried atan.__
3. A tan is an inverse trigonometric function. more can be found about atan on 
        * https://www.tensorflow.org/api_docs/python/tf/math/tan
        * https://www.varsitytutors.com/hotmath/hotmath_help/topics/tangent-function
        * https://www.youtube.com/watch?v=qZ6E3xrElGE